In [1]:
import sys
sys.path.append('../external/tslib')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import csv
import os
import time

random.seed(42)
np.random.seed(42)

### Preprocessing

In [7]:
# 读入CSV
df = pd.read_csv('../data/request_timeseries_all.csv')

# 把 timestamp 转成 date
df['date'] = pd.to_datetime(df['timestamp'], unit='s')

# 删除原来的 timestamp
df = df.drop(columns=['timestamp'])

# 把 date 列放到最前面
df = df[['date', 'requests']]

# roll up evry 10 data points
group_size = 10
grouped = df.groupby(df.index // group_size)

# 构建结果 DataFrame：保留起始时间，计算归一化请求数
result = grouped.agg({
    "date": "first",
    "requests": lambda x: x.sum() / 120
})

result.to_csv('../data/request_timeseries_aggregated.csv', index=False)

In [4]:
# 分割数据集，将数据集分为训练集和测试集，后30%作为测试集

df = pd.read_csv('../data/request_timeseries_all.csv')
train_size = int(len(df) * 0.7)
train_data = df[:train_size]
test_data = df[train_size:]
train_data.to_csv('../data/request_timeseries_train.csv', index=False)
test_data.to_csv('../data/request_timeseries_test.csv', index=False)

### Experiment

In [2]:
# 内部用前70%作为训练集
from lib.forecast.tslib_util import (
    parse_tslib_args,
    run_rslib_training,
)

In [ ]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'TimesNet',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--d_model', '16',
        '--d_ff', '32',
        '--des', 'Exp',
        '--itr', '1',
        '--top_k', '5',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_TimesNet_custom_ftS_sl36_ll18_pl36_dm16_nh8_el2_dl1_df32_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 0.8006218
	speed: 0.1288s/iter; left time: 273.2552s
Epoch: 1 cost time: 23.77434515953064
Epoch: 1, Steps: 185 | Train Loss: 1.0592931 Vali Loss: 0.5712931 Test Loss: 1.1151127
Validation loss decreased (inf --> 0.571293).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.3163972
	speed: 0.2978s/iter; left time: 576.4578s
Epoch: 2 cost time: 25.94798994064331
Epoch: 2, Steps: 185 | Train Loss: 0.4102752 Vali Loss: 0.3802178 Test Loss: 0.8637551
Validation loss decreased (0.571293 --> 0.380218).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.2887527
	speed: 0.3616s/iter; left time: 633.2391s
Epoch: 3 cost time: 28.733076333999634
Epoch: 3, Steps: 185 | Train Loss: 0.

In [8]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Transformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--d_model', '16',
        '--d_ff', '32',
        '--des', 'Exp',
        '--top_k', '5',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Transformer_custom_ftS_sl36_ll18_pl36_dm16_nh8_el2_dl1_df32_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 1.1000764
	speed: 0.0419s/iter; left time: 88.7785s
Epoch: 1 cost time: 6.521451950073242
Epoch: 1, Steps: 185 | Train Loss: 0.9944567 Vali Loss: 0.9976652 Test Loss: 1.5601455
Validation loss decreased (inf --> 0.997665).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.8269493
	speed: 0.0728s/iter; left time: 141.0362s
Epoch: 2 cost time: 6.203496932983398
Epoch: 2, Steps: 185 | Train Loss: 0.7940845 Vali Loss: 0.8185077 Test Loss: 1.2571694
Validation loss decreased (0.997665 --> 0.818508).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.6932233
	speed: 0.0759s/iter; left time: 132.9599s
Epoch: 3 cost time: 5.880542516708374
Epoch: 3, Steps: 185 | Train Loss: 0

In [9]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'PatchTST',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--n_heads', '2',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_PatchTST_custom_ftS_sl36_ll18_pl36_dm512_nh2_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 0.4372929
	speed: 0.0284s/iter; left time: 60.2718s
Epoch: 1 cost time: 5.059880018234253
Epoch: 1, Steps: 185 | Train Loss: 0.6148645 Vali Loss: 0.3760077 Test Loss: 0.8768855
Validation loss decreased (inf --> 0.376008).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.6021522
	speed: 0.0590s/iter; left time: 114.2012s
Epoch: 2 cost time: 3.9839162826538086
Epoch: 2, Steps: 185 | Train Loss: 0.4755860 Vali Loss: 0.3093416 Test Loss: 0.7799963
Validation loss decreased (0.376008 --> 0.309342).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.3138501
	speed: 0.0607s/iter; left time: 106.2064s
Epoch: 3 cost time: 4.909976005554199
Epoch: 3, Steps: 185 | Train Loss: 

In [11]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'DLinear',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_DLinear_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 1.2230064
	speed: 0.0113s/iter; left time: 23.8689s
Epoch: 1 cost time: 2.004391670227051
Epoch: 1, Steps: 185 | Train Loss: 1.4279965 Vali Loss: 1.3468503 Test Loss: 1.8771271
Validation loss decreased (inf --> 1.346850).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.8658008
	speed: 0.0292s/iter; left time: 56.4940s
Epoch: 2 cost time: 1.907482624053955
Epoch: 2, Steps: 185 | Train Loss: 0.9371984 Vali Loss: 0.9505950 Test Loss: 1.4296117
Validation loss decreased (1.346850 --> 0.950595).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.6315420
	speed: 0.0293s/iter; left time: 51.3705s
Epoch: 3 cost time: 1.980422019958496
Epoch: 3, Steps: 185 | Train Loss: 0.73

In [12]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Informer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)


Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Informer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 0.1946240
	speed: 0.0780s/iter; left time: 165.5439s
Epoch: 1 cost time: 13.941500902175903
Epoch: 1, Steps: 185 | Train Loss: 0.3166234 Vali Loss: 0.2460666 Test Loss: 0.4484399
Validation loss decreased (inf --> 0.246067).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2216434
	speed: 0.1718s/iter; left time: 332.6259s
Epoch: 2 cost time: 14.422443866729736
Epoch: 2, Steps: 185 | Train Loss: 0.1861564 Vali Loss: 0.2082187 Test Loss: 0.4138702
Validation loss decreased (0.246067 --> 0.208219).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1117789
	speed: 0.1780s/iter; left time: 311.6919s
Epoch: 3 cost time: 14.445901870727539
Epoch: 3, Steps: 185 | Train Los

In [13]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'FEDformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)

Using GPU
Use GPU: cuda:0
fourier enhanced block used!
modes=32, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
fourier enhanced block used!
modes=32, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
 fourier enhanced cross attention used!
modes_q=27, index_q=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
modes_kv=18, index_kv=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
>>>>>>>start training : long_term_forecast_ts_36_36_FEDformer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 0.2888545
	speed: 0.4068s/iter; left time: 862.7606s
Epoch: 1 cost time: 74.45337915420532
Epoch: 1, Steps: 185 | Train Loss: 0.6620080 Vali Loss: 0.2941423 Test Loss: 0.6104767
Validation loss decreased (inf --> 0.294142).  Saving model ...
U

In [14]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Autoformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)

Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Autoformer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 0.9664448
	speed: 0.1075s/iter; left time: 228.0724s
Epoch: 1 cost time: 20.011178016662598
Epoch: 1, Steps: 185 | Train Loss: 0.9055547 Vali Loss: 0.9944144 Test Loss: 1.3403921
Validation loss decreased (inf --> 0.994414).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.3069704
	speed: 0.2477s/iter; left time: 479.5522s
Epoch: 2 cost time: 20.3741512298584
Epoch: 2, Steps: 185 | Train Loss: 0.3341758 Vali Loss: 0.8342937 Test Loss: 1.1689628
Validation loss decreased (0.994414 --> 0.834294).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.2127078
	speed: 0.2525s/iter; left time: 442.0847s
Epoch: 3 cost time: 21.25951647758484
Epoch: 3, Steps: 185 | Train Loss

In [15]:
args = parse_tslib_args(
    [
        '--root_path', '../data/',
        '--data_path', 'request_timeseries_aggregated.csv',
        '--model_id', 'ts_36_36',
        '--model', 'Crossformer',
        '--data', 'custom',
        '--features', 'S',
        '--target', 'requests',
        '--freq', 'h',
        '--seq_len', '36',
        '--label_len', '18',
        '--pred_len', '36',
        '--e_layers', '2',
        '--d_layers', '1',
        '--factor', '3',
        '--enc_in', '1',
        '--dec_in', '1',
        '--c_out', '1',
        '--des', 'Exp',
        '--train_epochs', '12',
        '--use_gpu', 'True'
    ]
)

run_rslib_training(args)

Using GPU
Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_ts_36_36_Crossformer_custom_ftS_sl36_ll18_pl36_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 5912
val 821
test 1674
	iters: 100, epoch: 1 | loss: 0.2439458
	speed: 0.1314s/iter; left time: 278.6471s
Epoch: 1 cost time: 23.12880825996399
Epoch: 1, Steps: 185 | Train Loss: 0.3973501 Vali Loss: 0.2470131 Test Loss: 0.4445595
Validation loss decreased (inf --> 0.247013).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1630995
	speed: 0.2833s/iter; left time: 548.3990s
Epoch: 2 cost time: 22.804141759872437
Epoch: 2, Steps: 185 | Train Loss: 0.2060855 Vali Loss: 0.2432409 Test Loss: 0.5317065
Validation loss decreased (0.247013 --> 0.243241).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1522927
	speed: 0.2699s/iter; left time: 472.6744s
Epoch: 3 cost time: 20.46946430206299
Epoch: 3, Steps: 185 | Train Lo

### Forecast

In [ ]:
import numpy as np
from modelling.forecast.tslib_util import (
    TimeseriesTransformer,
    TimeseriesForecaster,
)

forecaster = TimeseriesForecaster()
forecaster.setTransformer(    
    transformer=TimeseriesTransformer(
        date_start=1734872400, date_scale=120, scale=True,
        scale_mean=3.69210541, scale_std=2.06017052
    )
)


Using cpu or mps
Use CPU


In [2]:
forecaster.forecast(
        enc_stamp = np.arange(0, 360, 10),
        enc_data = np.array([
            5.283333333333333,
            5.033333333333333,
            5.158333333333333,
            5.15,
            5.791666666666667,
            6.341666666666667,
            5.583333333333333,
            4.858333333333333,
            5.083333333333333,
            4.641666666666667,
            3.375,
            3.15,
            2.1333333333333333,
            1.8833333333333333,
            1.6,
            1.8833333333333333,
            1.4166666666666667,
            1.1416666666666666,
            0.675,
            0.4166666666666667,
            0.6416666666666667,
            0.9666666666666667,
            1.2416666666666667,
            1.1916666666666667, 
            1.1083333333333334,
            1.0916666666666666,
            0.6833333333333333,
            0.8083333333333333,
            0.8083333333333333,
            0.6916666666666667,
            0.48333333333333334,
            0.6333333333333333,
            0.7833333333333333,
            0.6333333333333333,
            1.1583333333333334,
            2.05
        ])
    )

array([2.2116888, 2.3310254, 2.523052 , 2.9497333, 2.812151 , 3.1983502,
       3.392816 , 3.762971 , 3.9305124, 4.118539 , 4.009357 , 4.2593546,
       4.32912  , 4.482236 , 4.4582357, 4.4998403, 4.4542484, 4.6175976,
       4.311565 , 4.472253 , 4.666833 , 4.3761573, 4.564841 , 4.3212533,
       4.263093 , 4.519063 , 4.6639857, 5.0408907, 4.552373 , 4.7632866,
       5.0331054, 5.0945263, 5.279776 , 5.267578 , 5.5426326, 5.5754967],
      dtype=float32)